<a href="https://colab.research.google.com/github/jchen6727/colab/blob/master/plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# clone colab
!git clone https://github.com/jchen6727/colab.git

fatal: destination path 'colab' already exists and is not an empty directory.


In [ ]:
# import and filestrings
import pandas
import plotly.express as px

import argparse

bfiles = ['colab/bopt/b{}.csv'.format(i) for i in ['011', '101', '110', '111']]
ofiles = ['colab/optuna/o{}.csv'.format(i) for i in ['011', '101', '110', '111']]

print(bfiles)
print(ofiles)


['colab/bopt/b011.csv', 'colab/bopt/b101.csv', 'colab/bopt/b110.csv', 'colab/bopt/b111.csv']
['colab/optuna/o011.csv', 'colab/optuna/o101.csv', 'colab/optuna/o110.csv', 'colab/optuna/o111.csv']


In [ ]:
# load csv to pandas
df = pandas.read_csv('colab/bopt/b111.csv')

In [ ]:
# create additional columns (min)
df['min_loss'] = df['loss'].expanding().min()

actual = { # weights from cfg, AMPA, GABA, NMDA
    'PYR->BC_AMPA' : 0.36e-3, "BC->BC_GABA"  : 4.5e-3 , "PYR->BC_NMDA" : 1.38e-3 ,
    'PYR->OLM_AMPA': 0.36e-3, "BC->PYR_GABA" : 0.72e-3, "PYR->OLM_NMDA": 0.7e-3  ,
    'PYR->PYR_AMPA': 0.02e-3, "OLM->PYR_GABA": 72e-3  , "PYR->PYR_NMDA": 0.004e-3,
    'PYR': 3.34,
    'BC': 19.7,
    'OLM': 3.47
}

rename = {
       'config/netParams.connParams.BC->BC_GABA.weight',
       'config/netParams.connParams.BC->PYR_GABA.weight',
       'config/netParams.connParams.OLM->PYR_GABA.weight',
       'config/netParams.connParams.PYR->BC_AMPA.weight',
       'config/netParams.connParams.PYR->BC_NMDA.weight',
       'config/netParams.connParams.PYR->OLM_AMPA.weight',
       'config/netParams.connParams.PYR->OLM_NMDA.weight',
       'config/netParams.connParams.PYR->PYR_AMPA.weight',
       'config/netParams.connParams.PYR->PYR_NMDA.weight',
}

rename = { n: n.split('.')[2] for n in rename if n in df.columns}

params = rename.values()

df.rename(rename, axis=1, inplace=True)

def expanding_idxmin(df, col):
    curr = 0
    for i in range(len(df)):
        if df[col][i] < df[col][0:i].min():
            yield i
            curr = i
        else:
            yield curr

for param in params:
    df['MIN_{}'.format(param)] = df[param][expanding_idxmin(df, 'loss')].reset_index(drop=True)

for param in ['PYR', 'BC', 'OLM']:
    df['MIN_{}'.format(param)] = df[param][expanding_idxmin(df, 'loss')].reset_index(drop=True)


In [ ]:
deltas = df[df['min_loss'].diff() != 0].index.tolist()

In [ ]:
# show plots
fig = px.scatter(df, x=df.index, y='loss')
fig.add_scatter(x=df.index, y=df['min_loss'], mode='lines', name='min_loss')
for delta in deltas:
    fig.add_vline(delta)
fig.show()
################################################################################
#
#  LOSS (MSE) PER TRIAL RUN {RED: BEST}
#
################################################################################

In [ ]:
# show plots

figs = []
for pop in ['BC', 'OLM', 'PYR']:
    fig = px.scatter(df, x=df.index, y='PYR')
    fig.add_scatter(x=df.index, y=df['MIN_PYR'], mode='lines', name='MIN_PYR')
    fig.add_hline(actual['PYR'])
    for delta in deltas:
        fig.add_vline(delta)
    figs.append(fig)

for fig in figs:
    fig.show()
################################################################################
#
#  FREQUENCY PER TRIAL RUN {BLUE: ACTUAL, RED: BEST}
#
################################################################################

In [ ]:
figs = []
for cfg in params:
    fig = px.scatter(df, x=df.index, y=cfg)
    fig.add_scatter(x=df.index, y=df['MIN_{}'.format(cfg)], mode='lines', name='MIN_{}'.format(cfg))
    fig.add_hline(actual[cfg])
    for delta in deltas:
        fig.add_vline(delta)
    figs.append(fig)

In [ ]:
for fig in figs:
    fig.show()